# 本チャプターの目次
1. テクニカルメタデータについて
2. テクニカルメタデータの一種データプロファイリング
3. PySparkでデータプロファイリングをしてみよう1 
4. PySparkでデータプロファイリングをしてみようその２
5. PySparkでデータプロファイリングをしてみようその3
6. データプロファイリングの結果をテーブルに格納してみよう
7. データ品質

# テクニカルメタデータについて
テクニカルメタデータとは、

# テクニカルメタデータの一種データプロファイリング


In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
# メタデータ取得対象のデータを取得する
# 前のチャプターで「データベース名」「テーブル名」「テーブル定義」「サマリー」を登録したテーブルを取得します

df=spark.sql("select * from  metadata_tmp.sample_metadata")


# PySparkでデータプロファイリングをしてみようその１

データプロファイリングその１では、以下の3つを算出してみます。  
3つのうち件数をrow_num(件数)をメタデータストアに保存していきます。

- 件数
- 平均
- 合計値

## 件数/平均/合計の使い道
例えば件数はよく、異常な状態を見付けることに使えます。  
例えば、前日までに１０件だったものが急に1000件になったらアラートを鳴らすということにも使えたりします。  
データ基盤は時には何万というテーブルが存在しているため、一つ一つ目で見るのは不可能に近いのが現実です。  
そこで件数のような簡単なチェックを入れておくことでも、テーブル数が増え続けても問題ないデータ基盤を作ることが可能です。  

一方で、取得しやすいがゆえに何も考えずに取得することは無意味になりますので、しっかりと自身の環境に適用できるかどうかを判断することは大切になります。  
例えば、年齢の合計値はあまり意味を成さないかもしれませんし時と場合によっては、合計を知ることで何か得るものがあるかもしれません。  

In [ ]:
# 件数の取得
spark.sql("REFRESH TABLE data_management_cruch_course.jinko_table ")
row_num=spark.sql("select count(*) as row_num from data_management_cruch_course.jinko_table ")
row_num.show()

In [ ]:
# 平均と合計値
# 今回は平均として、女性と男性の合計値の平均をとっていこうと思います。
spark.sql("REFRESH TABLE data_management_cruch_course.jinko_table ")
sum=spark.sql("select cast(sum(jinko_male)+sum(jinko_femail) as decimal(19, 0)) as sum from data_management_cruch_course.jinko_table ")
# decimalにcastしないと数値が指数表記になってみづらくなるのでdecimalにしています。
sum.show()

In [ ]:
# 平均と合計値
# 今回は平均として、女性と男性の合計値の平均をとっていこうと思います。
spark.sql("REFRESH TABLE data_management_cruch_course.jinko_table ")
avg=spark.sql("select avg(sum) as avg from (select cast(sum(jinko_male)+sum(jinko_femail) as decimal(19, 0)) as sum from data_management_cruch_course.jinko_table) ")
avg.show()

# PySparkでデータプロファイリングをしてみようその２
データプロファリングその２では、以下の２つを算出していきます。

- カーディナリティ
- セレクティビティ

# カーディナリティの使い道
カーディナリティはスモールファイル問題やデータスキューネスを発見するための手段としてビッグデータ基盤では使われます。  
データスキューネスとは、データに偏りがある状態を指します。  

ビッグデータ基盤における偏りやスモールファイルは非常に問題になることがあります。  
例えば、クエリのエラーを多発させたり、データの転送を遅くさせたりするする原因にもなります。  

スモールファイル問題について詳しく知りたい方は是非以下のコースも受講してみてください。  
https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB   

# セレクティビティの使い道
セレクティビティとは、検索した時に結果が何件返却されるか？というものです  

セレクティビティは、クエリのしやすさに直結してきます。  
データを探索する際に、重複したデータが出てきたらどうでしょうか(そもそも重複に気付けない可能性がありますが)？  
重複を除く処理をSQLに記載しなければなりませんし、さらには  
最終的な分析結果を間違えることもあります。たとえば、結果が2倍になってしまうかも。  
数億というレコードの中で重複を分析の結果だけ見て判断するというのはできないものです。  
そうなれば、再度モデルの作成仕直しや開発のやり直しをしなければならなくなります。  

ビッグデータ基盤にはPK（プライマリーキー）のような仕組みを持っているものが少なく、データの重複が多々発生してしまいます。  

そこでセレクティビティのチェックを行うことによって、重複しているデータを見つけ出し対処を行うことができるのです※。  

※どのような対処を取るべきなのか？についてはデータ品質管理という方法があります。準備が出来次第別のコースで作成予定です。  

# PySparkでデータプロファイリングをしてみようその3

# データプロファイリングの結果をMysqlテーブルに格納してみよう

頑張ればSparkを使って書き込むこともできますが、きれいなPGにはならないのでSparkの機能は使わずにPythonから実行するようにします




# データ品質